In [1]:
%reload_ext autoreload
%autoreload 2


In [2]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle
import pandas as pd
from subprocess import Popen,PIPE,STDOUT,call 
from utils import datasets

BDD100K_DIRECTORY = os.path.join('/media/dean/datastore1/datasets/BerkeleyDeepDrive', 'bdd100k')
WORKING_DIRECTORY = '/media/dean/datastore1/datasets/darknet'

TRAINERS_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'trainers')
ANNOTATIONS_FILE = os.path.join(BDD100K_DIRECTORY, 'labels/bdd100k_labels_images_train.json')

CONSTRUCTION_ANNOTATIONS_FILE = os.path.join('/media/dean/datastore1/datasets/annotation_files', 'Kache-Logs--Tanveer--7_40_corrected.json')

CONSTRUCTION_ANNOTATIONS_FILE2 = os.path.join('/media/dean/datastore1/datasets/annotation_files', 'Kache-Logs--Tanveer--6_40_corrected.json')
VAL_ANNOTATIONS_FILE = os.path.join(BDD100K_DIRECTORY, 'labels/bdd100k_labels_images_val.json')
COCO_ANNOTATIONS_FILE = os.path.join('/media/dean/datastore/datasets/road_coco/darknet/data/coco/annotations/instances_train2014.json')

BASE_DATA_CONFIG = os.path.join(WORKING_DIRECTORY, 'cfg', 'bdd100k.data')
BASE_MODEL_CONFIG = os.path.join(WORKING_DIRECTORY, 'cfg', 'yolov3-bdd100k.cfg')

In [3]:
## Prepare Dataset ##



construction_set = datasets.DataFormatter(annotations_list = CONSTRUCTION_ANNOTATIONS_FILE, input_format = datasets.Format.bdd,
                                 output_path = os.path.join(WORKING_DIRECTORY,'data/construction_set'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/', convert_attributes = True)



Saving to Pickle File: annotation_files_Kache-Logs--Tanveer--7_40_corrected.json.pickle
Length of COCO Images 1000
Length of COCO Annotations: 11778
loading annotations into memory...
Done (t=0.10s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########

BIKE | Annotations: 1  | Images:  1
BUS | Annotations: 30  | Images:  30
CAR | Annotations: 3834  | Images:  949
CONSTRUCT-BARREL | Annotations: 2443  | Images:  279
CONSTRUCT-CONE | Annotations: 188  | Images:  37
CONSTRUCT-EQUIPMENT | Annotations: 154  | Images:  71
CONSTRUCT-POLE | Annotations: 488  | Images:  127
CONSTRUCT-SIGN | Annotations: 219  | Images:  123
MOTOR | Annotations: 21  | Images:  18
PERSON | Annotations: 215  | Images:  51
RIDER | Annotations: 14  | Images:  13
TRAFFIC LIGHT | Annotations: 76  | Images:  30
TRAFFIC SIGN | Annotations: 2255  | Images:  734
TRAFFIC SIGN-SLOW_SIGN | Annotations: 1  | Images:  1
TRAFFIC SIGN-SPEED_SIGN | Annotations: 32  | Images:  28
TRAFFIC SIGN-STOP_SIGN | 

In [4]:
## Prepare Dataset ##



construction_set2 = datasets.DataFormatter(annotations_list = CONSTRUCTION_ANNOTATIONS_FILE2, input_format = datasets.Format.bdd,
                                 output_path = os.path.join(WORKING_DIRECTORY,'data/construction_set'),
                                 trainer_prefix = 'COCO_train2014_0000', convert_attributes = True,
                                 s3_bucket = 'kache-scalabel/kache_ai/frames/')



Saving to Pickle File: annotation_files_Kache-Logs--Tanveer--6_40_corrected.json.pickle
Length of COCO Images 1000
Length of COCO Annotations: 12958
loading annotations into memory...
Done (t=0.11s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########

BIKE | Annotations: 9  | Images:  4
BUS | Annotations: 6  | Images:  5
CAR | Annotations: 4108  | Images:  939
CONSTRUCT-BARREL | Annotations: 2908  | Images:  283
CONSTRUCT-CONE | Annotations: 223  | Images:  38
CONSTRUCT-EQUIPMENT | Annotations: 191  | Images:  117
CONSTRUCT-POLE | Annotations: 569  | Images:  143
CONSTRUCT-SIGN | Annotations: 279  | Images:  162
MOTOR | Annotations: 13  | Images:  12
PERSON | Annotations: 217  | Images:  57
RIDER | Annotations: 7  | Images:  7
TRAFFIC LIGHT | Annotations: 137  | Images:  39
TRAFFIC SIGN | Annotations: 1955  | Images:  695
TRAFFIC SIGN-SLOW_SIGN | Annotations: 2  | Images:  2
TRAFFIC SIGN-SPEED_SIGN | Annotations: 92  | Images:  82
TRAFFIC SIGN-STOP_SIGN | An

In [5]:
construction_set.merge(construction_set2, include=['CONSTRUCT-BARREL','CONSTRUCT-CONE','CONSTRUCT-EQUIPMENT', 'CONSTRUCT-POLE', 'CONSTRUCT-SIGN', 'person', 'train', 'truck', 'car', 'traffic light', 'traffic light-red','traffic light-amber','traffic light-green', 'bus', 'bike', 'motor', 'traffic sign','traffic sign-slow_sign', 'traffic sign-speed_sign', 'traffic sign-stop_sign'], reject_new_categories = False)

Successfully merged 940 images | and  12066 annotations
Length of COCO Annotations: 23844
loading annotations into memory...
Done (t=0.13s)
creating index...
index created!

########### DATASET DISTRIBUTION: ###########

BIKE | Annotations: 10  | Images:  5
BUS | Annotations: 35  | Images:  34
CAR | Annotations: 7628  | Images:  1828
CONSTRUCT-BARREL | Annotations: 5200  | Images:  547
CONSTRUCT-CONE | Annotations: 411  | Images:  75
CONSTRUCT-EQUIPMENT | Annotations: 334  | Images:  181
CONSTRUCT-POLE | Annotations: 1036  | Images:  264
CONSTRUCT-SIGN | Annotations: 482  | Images:  275
MOTOR | Annotations: 27  | Images:  23
PERSON | Annotations: 425  | Images:  103
RIDER | Annotations: 14  | Images:  13
TRAFFIC LIGHT | Annotations: 212  | Images:  68
TRAFFIC SIGN | Annotations: 4113  | Images:  1393
TRAFFIC SIGN-SLOW_SIGN | Annotations: 2  | Images:  2
TRAFFIC SIGN-SPEED_SIGN | Annotations: 121  | Images:  107
TRAFFIC SIGN-STOP_SIGN | Annotations: 5  | Images:  5
TRAILER | Annotations

In [6]:
construction_set.export(format = datasets.Format.bdd, paginate = False)

In [9]:
annotations_file = '/media/dean/datastore1/datasets/darknet/data/construction_set/bdd100k/annotations/bdd100k_altered_annotations.json'
!cd $WORKING_DIRECTORY && python3 darkernet.py -b 'kache-scalabel/kache_ai/frames/' -a $annotations_file -f BDD -r True

In [10]:
# KACHE_S3 = 'kache-scalabel/kache_ai/frames/'
# annotations_file = os.path.join(WORKING_DIRECTORY,'data/kache_set/annotations/Kache-Logs--Tanveer-WithTrafficSigns_partial_complete_1.json')
# !cd $WORKING_DIRECTORY && python3 darkernet.py -a $annotations_file -f BDD -b $KACHE_S3 -r True

In [11]:
# WORKING_DIR = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/'
# kache_set = datasets.DataFormatter(annotations_list = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/kacheai_logs.pickle', image_list = '/media/dean/datastore1/datasets/kache_ai/frames/kache_set/', input_format = datasets.Format.kache,
#                                  output_path = os.path.join(WORKING_DIR,'scalabel_files'), use_cache = True,
#                                  trainer_prefix = 'COCO_train2014_0000', pickle_file = '/media/dean/datastore1/datasets/Scripts/kache_ai_frames_kache_set_kacheai_logs.pickle.pickle',
#                                  s3_bucket = 'kache-scalabel/kache_ai/frames/', check_s3 = False, use_static_categories = True)


In [12]:
# kache_set.export(format = datasets.Format.scalabel, paginate = True)

In [13]:
# cocodd_set.merge(coco_set, include=['person', 'train', 'truck', 'car', 'traffic light', 'bus'], reject_new_categories = True)

In [14]:
# cocodd_set.export(format = datasets.Format.bdd, paginate = False)